In [4]:
import numpy as np 
import abc

In [69]:
############################################
# Abstract Class Derivative
############################################

class Derivative(abc.ABC):
    def __init__(self, seed=0):
        self.seed = seed
        self.name = 'abstract'
        self.params = {}

    @abc.abstractmethod
    def price_monte_carlo(self):
        raise NotImplementedError

    @abc.abstractmethod
    def greeks_difference_method(self):
        raise NotImplementedError
    
    @abc.abstractmethod
    def greeks_exact(self):
        raise NotImplementedError

    @abc.abstractmethod
    def greeks_malliavin(self, x, y, T):
        raise NotImplementedError

In [76]:
class VanillaDerivative(Derivative):
    def __init__(self, S0, K, r, sigma, T, payoff, name, seed=0):
        self.seed = seed
        self.name = "_".join(['vanilla',"name"])
        self.params = {'price_0':S0,'strike':K,'interest_rate':r,'vol':sigma,'maturity':T}
        self.payoff = payoff
    
    def price_monte_carlo(self,N,epsilon = 0,param__=None):
        G = np.random.normal(size=N)
        sum = 0
        params__ = self.params.copy()
        if param__:
            params__[param__] = self.params[param__] + epsilon
        for i in range(N):
            sum += self.payoff(params__['price_0'] * np.exp((params__['interest_rate']-params__['vol']**2/2)*params__['maturity']+params__['vol']*(params__['maturity']**0.5)*G[i]))

        return sum*np.exp(-params__['interest_rate']*params__['maturity'])/N
    
    def greeks_difference_method(self,N,epsilon,param__,order = 1):
        if order == 1 :
            return (self.price_monte_carlo(N,epsilon,param__) + self.price_monte_carlo(N,-epsilon,param__) - 2* self.price_monte_carlo(N))/(epsilon*2)
        if order == 2 : 
            return (self.price_monte_carlo(N,epsilon,param__) - self.price_monte_carlo(N,-epsilon,param__))/(epsilon**2)
    
    def greeks_exact(self):
        raise NotImplementedError

    def greeks_malliavin(self, x, y, T):
        raise NotImplementedError

In [77]:
T = 1
S0 = 100
sigma = 0.2
r = 0.05
K = 140
payoff = lambda x: 0 if x <= K else 1
name = 'digital'

In [78]:
call_option = VanillaDerivative(S0, K, r, sigma, T, payoff, name)

In [79]:
call_option.greeks_difference_method(N=100_000,epsilon=0.05,param__='price_0',order=2)

0.09131802475206839